Copyright **`(c)`** 2022 Giovanni Squillero `<squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  


# Lab 3: ES

## Task

Write agents able to play [*Nim*](https://en.wikipedia.org/wiki/Nim), with an arbitrary number of rows and an upper bound $k$ on the number of objects that can be removed in a turn (a.k.a., *subtraction game*).

The goal of the game is to **avoid** taking the last object.

* Task2.1: An agent using fixed rules based on *nim-sum* (i.e., an *expert system*)
* Task2.2: An agent using evolved rules using ES

## Instructions

* Create the directory `lab2` inside your personal course repository for the course 
* Put a `README.md` and your solution (all the files, code and auxiliary data if needed)

## Notes

* Working in group is not only allowed, but recommended (see: [Ubuntu](https://en.wikipedia.org/wiki/Ubuntu_philosophy) and [Cooperative Learning](https://files.eric.ed.gov/fulltext/EJ1096789.pdf)). Collaborations must be explicitly declared in the `README.md`.
* [Yanking](https://www.emacswiki.org/emacs/KillingAndYanking) from the internet is allowed, but sources must be explicitly declared in the `README.md`.



In [10]:
import logging
from pprint import pprint, pformat
from collections import namedtuple
import random
from copy import deepcopy
from tqdm import tqdm


## The *Nim* and *Nimply* classes

In [11]:
Nimply = namedtuple("Nimply", "row, num_objects")


In [12]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        if k is None:
            k = (num_rows - 1) * 2 + 1
        self._k = k

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    @property
    def k(self) -> int:
        return self._k

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects


## Strategies

In [13]:
def pure_random(state: Nim) -> Nimply:
    """A completely random move"""
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, min(state.rows[row], state.k))
    return Nimply(row, num_objects)


In [15]:
def gabriele(state: Nim) -> Nimply:
    """Pick always the maximum possible number of the lowest row"""
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, min(state.k + 1, c + 1))]
    return Nimply(*max(possible_moves, key=lambda m: (-m[0], m[1])))


In [16]:
def take_half_from_last_row(state: Nim) -> Nimply:
    row = -1
    for i in range(len(state.rows) -1, -1, -1):
        if state.rows[i] > 0:
            row = i
            break
    n_obj = min(state.k, state.rows[row] // 2) if state.rows[row] // 2 != 0 else min(state.k, state.rows[row])
    return Nimply(row, n_obj)

In [17]:
def take_half_from_top(state: Nim) -> Nimply:
    row = -1
    for i in range(len(state.rows)):
        if state.rows[i] > 0:
            row = i
            break
    n_obj = min(state.k, state.rows[row] // 2) if state.rows[row] // 2 != 0 else min(state.k, state.rows[row])
    return Nimply(row, n_obj)


In [19]:
def take_one(state: Nim) -> Nimply:
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    return Nimply(row, 1)

In [20]:
def take_even_odd(state: Nim) -> Nimply:
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    if row % 2 == 0:
        n_obj = min(state.k, state.rows[row] // 2)
    else:
        n_obj = max(1, min(state.k, state.rows[row]))
    return Nimply(row, n_obj)

In [21]:
def leave_one(state: Nim) -> Nimply:
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    c = state.rows[row]
    n_obj = max(min(state.k, c -1), 1)
    return Nimply(row, n_obj)


In [22]:
def adaptive(state: Nim) -> Nimply:
    """A strategy that can adapt its parameters"""
    genome = {"love_small": 0.5}


In [23]:
import numpy as np


def nim_sum(state: Nim) -> int:
    tmp = np.array([tuple(int(x) for x in f"{c:032b}") for c in state.rows])
    xor = tmp.sum(axis=0) % 2
    return int("".join(str(_) for _ in xor), base=2)


def analize(raw: Nim) -> dict:
    cooked = dict()
    cooked["possible_moves"] = dict()
    for ply in (Nimply(r, o) for r, c in enumerate(raw.rows) for o in range(1, min(raw.k + 1, c + 1))):
        tmp = deepcopy(raw)
        tmp.nimming(ply)
        cooked["possible_moves"][ply] = nim_sum(tmp)
    return cooked


def optimal(state: Nim) -> Nimply:
    analysis = analize(state)
    logging.debug(f"analysis:\n{pformat(analysis)}")
    spicy_moves = [ply for ply, ns in analysis["possible_moves"].items() if ns != 0]
    if not spicy_moves:
        spicy_moves = list(analysis["possible_moves"].keys())
    ply = random.choice(spicy_moves)
    return ply


## Oversimplified match

In [24]:
def game(n_rows, k, strategy):
    # logging.getLogger().setLevel(logging.INFO)

    strategies = (strategy, optimal)

    nim = Nim(n_rows, k)
    # logging.info(f"init : {nim}")
    player = 0
    while nim:
        ply = strategies[player](nim)
        # logging.info(f"ply: player {player} plays {ply}")
        nim.nimming(ply)
        # logging.info(f"status: {nim}")
        player = 1 - player
    # logging.info(f"status: Player {player} won!")
    return player


In [25]:
def compare_strategies(strategies, n_rows, k):
    NUM_GAMES = 1000
    for name, s in strategies.items():
        winners = [game(n_rows, k, s) for _ in range(NUM_GAMES)]
        avg_1 = (sum(winners)) / (NUM_GAMES / 100)
        print(f"Optimal wins: {avg_1}% against {name}")

In [88]:
compare_strategies({"even_odd": take_even_odd, "half_last": take_half_from_last_row, "one": take_one, "half_top": take_half_from_top}, 5, 3)

Optimal wins: 10.1% against even_odd
Optimal wins: 74.0% against half_last
Optimal wins: 71.7% against one
Optimal wins: 72.6% against half_top


In [26]:
class Player:
    def __init__(self, weights: tuple[float], strategies):
        self.weights = weights
        self.strategies = strategies

    def get_strategy(self):
        min_weight = min(self.weights)
        if min_weight <= 0:
            shifted_weights = [w - min_weight + 1.0 for w in self.weights]  # Adding 1 ensures non-zero weights
        else:
            shifted_weights = self.weights  # No need to shift if all weights are non-negative
        return random.choices(self.strategies, weights=shifted_weights, k=1)[0]

    def __str__(self):
        w_to_s = [f'{w} ' for w in self.weights]
        return ''.join(w_to_s)


In [27]:
rules = [take_one, take_even_odd, take_half_from_top, take_half_from_last_row, gabriele, pure_random]

In [28]:
def generate_population(size = 30):
    pop = [Player(tuple([random.random() for _ in range(len(rules))]), rules) for _ in range(size)]
    return pop

In [29]:
def offspring(parents, mutation, size = 210):
    os = []
    parent_subset = [random.choice(parents) for _ in range(size)]
    for i in range(size):
        weights = np.array(parent_subset[i].weights)
        mutations = np.array([random.gauss(0, mutation) for _ in range(weights.shape[0])])
        weights += mutations
        new_player = Player(tuple(weights), rules)
        os.append(new_player)
    return os

In [30]:
def game_simulation(n_rows: int, k: int, Player_1: Player, Player_2: Player):
    # logging.getLogger().setLevel(logging.INFO)
    players = (Player_1, Player_2)
    nim = Nim(n_rows, k)
    # logging.info(f"init : {nim}")
    player = 0
    while nim:
        ply = players[player].get_strategy()(nim)
        # logging.info(f"ply: player {player} plays {ply}")
        nim.nimming(ply)
        # logging.info(f"status: {nim}")
        player = 1 - player
    # logging.info(f"status: Player {player} won!")
    return player


In [31]:
NUM_GAMES = 25
ROWS = 5
K = 5

optimal_player = Player(tuple([1.0]), [optimal])
def fitness(individual: Player) -> float:
    winners = [game_simulation(ROWS, K, individual, optimal_player) for _ in range(NUM_GAMES)]
    return (NUM_GAMES - sum(winners)) / NUM_GAMES

## ( &mu;, &lambda; )-ES
<p>Self Adaptive Strategy I<p>

In [32]:
mu = 10
λ = 70 # 7 * mu rule
σ = 0.01
GENERATIONS = 500

In [33]:
def evaluate_offspring(offs: list[Player]) -> list[(Player, float)]:
    return [(play, fitness(play)) for play in offs]

In [ ]:
pop = generate_population(size = mu)
evals = evaluate_offspring(pop)
evals.sort(key = lambda x: x[1], reverse = True)
prev_evals = map(lambda  x: x[1], evals)
for man in pop:
    print(fitness(man), end=" ")
stats = [0,0]
for step in tqdm(range(GENERATIONS)):
    # generate offspring
    off = offspring(pop, σ, size = λ)
    #evaluate offspring
    evals = evaluate_offspring(off)
    # sort in decresing order by fitness value (games wins against optimal)
    evals.sort(key = lambda x: x[1], reverse = True)
    # retrieve new population
    pop = [p for p, v in evals[:mu]]
    # get only the fitness values in decresing order
    curr_evals = map(lambda  x: x[1], evals[:mu])
    # compute number of improvements related to the rank
    succs = sum([1 for c, p in zip(curr_evals, prev_evals) if c > p])
    stats[0] += succs
    stats[1] += mu
    prev_evals = curr_evals
    if (step + 1) % 20 == 0: # every 20 steps adjust mutation
        print(f"Max_fitness: {evals[0][1]}") # It's not totally deterministic, just to have an idea
        if stats[0] / stats[1] < 1 / 5:
            σ /= 1.1
        elif stats[0] / stats[1] > 1 / 5:
            σ *= 1.1
        stats = [0, 0]

In [42]:
for i, man in enumerate(pop):
    print(f"Player_{i}")
    print(f"weights: {man}", end="")
    print(f"fitness: {fitness(man)}")

with open("comma_weights.txt", "w") as f:
    for man in pop:
        f.write(f"{man}\n")

Player_0
weights: 0.8747547296753383 1.174081762774021 0.5637191911230309 0.6874056490574837 0.13658004993064574 0.07210579441188611 fitness: 0.32
Player_1
weights: 0.8986483868042027 1.1596811606090258 0.5711254375013113 0.688716283171604 0.14415686514616907 0.07236553069483193 fitness: 0.48
Player_2
weights: 0.8787514397816611 1.1772667474209966 0.5620133111260542 0.6976207351737103 0.14438583170262928 0.07251043671321984 fitness: 0.64
Player_3
weights: 0.8696481020087934 1.1853664589070303 0.5860150694060065 0.7008665893292456 0.14852250782686657 0.06456486419434918 fitness: 0.44
Player_4
weights: 0.8832967088400913 1.1795641526743654 0.5814677739090952 0.7135281356203919 0.15595055392425722 0.056620645151317234 fitness: 0.2
Player_5
weights: 0.8917591179935029 1.1874058844764541 0.6017936520859832 0.704996440913763 0.15613393341925183 0.07649012170316355 fitness: 0.48
Player_6
weights: 0.903109683895098 1.1721256561413615 0.5756139986623511 0.6938058106641718 0.14759821713910273 0.

In [43]:
games = 10_000
best_evolved = pop[0]
winners = [game_simulation(ROWS, K, best_evolved, optimal_player) for _ in tqdm(range(games))]
print((games - sum(winners)) / games)

100%|██████████| 10000/10000 [00:23<00:00, 421.93it/s]

0.4145


## ( &mu; + &lambda; )-ES
<p>Self Adaptive Strategy II<p>

In [41]:
mu = 10
λ = 70 # 7 * mu rule
σ = [0.01] * mu
GENERATIONS = 500

In [43]:
def offspring(parents, size = 210):
    os = []
    mut_list = []
    parent_subset = parents[:, np.random.randint(0, parents.shape[1], size=(size,))] # 2 x λ
    for i in range(size):
        weights = np.array(parent_subset[0, i].weights)
        tmp = random.gauss(parent_subset[1, i], 0.2)
        mutation = tmp if tmp > 1e-5 else 1e-5 # update mutation before update the weights (negative replaced with small value)
        mutations = np.array([random.gauss(0, mutation) for _ in range(weights.shape[0])])
        weights += mutations
        new_player = Player(tuple(weights), rules)
        os.append(new_player)
        mut_list.append(parent_subset[1, i])
    return np.array([[*os], [*mut_list]])

In [ ]:
pop = generate_population(size = mu)
evals = evaluate_offspring(pop)
evals.sort(key = lambda x: x[1], reverse = True)
prev_evals = map(lambda  x: x[1], evals)

pop = np.array([[*pop], [*σ], [*prev_evals]]) # Player, σ value shape: 3 x mu

for step in tqdm(range(GENERATIONS)):
    # generate offspring
    off = offspring(pop[0:2, :], size = λ)
    #evaluate offspring
    evals = np.array([f for p, f in evaluate_offspring([off[0, i] for i in range(off.shape[1])])])
    off = np.vstack([off, evals])
    # plus strategy
    pop = np.hstack((pop, off))
    fit_row = pop[-1,:] # last row is fitness
    pop = pop[:, np.argsort(fit_row)]
    if step % 20: # to reduce output size
        print(f"Max fitness: {pop[-1, -1]}") # not deterministic just to see if there is a global optimization
    pop = np.copy(pop[:, -mu:])


In [53]:
best_players = [pop[0, i] for i in range(pop.shape[1])]
for i, man in enumerate(best_players):
    print(f"Player_{i}")
    print(f"weights: {man}", end="")
    print(f"fitness: {fitness(man)}")

with open("plus_weights.txt", "w") as f:
    for man in best_players:
        f.write(f"{man}\n")

Player_0
weights: 0.02875673585701085 1.3651857268490717 1.1166753384812274 0.14077847236396054 0.05682521679939021 0.04595205914107282 fitness: 0.56
Player_1
weights: 0.02876197925628513 1.3651785906322917 1.1166625197458722 0.14076987701928537 0.05680499982084915 0.04597331298517471 fitness: 0.44
Player_2
weights: 0.018589278758053544 1.324757116084051 1.1205752831691425 0.0948172540605404 0.06777655282815388 0.03758948870208941 fitness: 0.4
Player_3
weights: 0.028789441968199346 1.3652130095990296 1.1166339074589187 0.1407455691723861 0.056800890808821385 0.046018989015716924 fitness: 0.48
Player_4
weights: 0.02863503179511835 1.3666905240104479 1.1170558786721734 0.13789318433934317 0.06187638645579206 0.046026793514681405 fitness: 0.56
Player_5
weights: 0.028781456921068514 1.3651981185355362 1.1166505518642302 0.14076095140772896 0.05681162357140301 0.0460403396578738 fitness: 0.48
Player_6
weights: 0.028759590880610853 1.3651852761376628 1.1166915568946236 0.14080256654305803 0.

In [54]:
games = 10_000
best_evolved = best_players[0]
winners = [game_simulation(ROWS, K, best_evolved, optimal_player) for _ in tqdm(range(games))]
print((games - sum(winners)) / games)

100%|██████████| 10000/10000 [00:22<00:00, 447.46it/s]

0.5313
